<a href="https://colab.research.google.com/github/getaccept/notebooks/blob/master/API_copy_template_to_entities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Copy templates using GetAccept *API*

Functions to copy one or many templates from a main entity to one or many subentities including editor blocks and content

__** NOTE ***__
- Currently library/uploaded videos are not supported, only video from URL
- Currently folders and resources are not supported, only editor templates
- PDF pages are currently not supported

In [ ]:
# import dependencies
import requests
import os
import io
import json
import copy
import mimetypes
import ipywidgets as widgets
from google.colab import files
from PIL import Image

#Constants 
BASE_URL = "https://api.getaccept.com/v1"
INT_BASE_URL = "https://int.getaccept.com"
destination_entities = []
SOURCE_ENTITY_ID = ""
last_header = ""
#@title ↓↓ Click here to start
#@markdown This step might take a few seconds to run. <br>
#@markdown Then use __shift+enter__ key or click ► left of each step to go through the flow

In [ ]:
email_widget = widgets.Text(
    value="",
    placeholder="Enter login email",
    description="Email:",
    disabled=False
)
password_widget = widgets.Password(
    value="",
    placeholder="Enter password",
    description="Password:",
    disabled=False
)
#@markdown Use the form below to fill in login details to your entity in GetAccept and then run next cell to login
widgets.VBox([email_widget, password_widget])

In [ ]:
#@markdown Login and store API token
if email_widget.value and password_widget.value:
  payload = { "email": email_widget.value, "password": password_widget.value}
  if SOURCE_ENTITY_ID != "":
    payload["entity_id"] = SOURCE_ENTITY_ID
  response = requests.post(BASE_URL+"/auth", json=payload)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  else:
    raise TypeError(data["errors"], "Please check your credentials")
  # Check login and list entities
  response = requests.get(BASE_URL+"/users/me", headers=source_auth_headers)
  user_data = response.json()
  print("Logged in as " + user_data["user"]["first_name"] + " on entity " + user_data["user"]["entity_name"])
  SOURCE_ENTITY_ID = user_data["user"]["entity_id"]
else:
  raise TypeError("Could not login, missing email or password!")

In [ ]:
#@markdown Select the source entity you would like to get template data from. When you're done, run the next cell
source_entity_list = list(map(lambda x: (x["name"],x["id"]), user_data["entities"]))
source_entity_picker = widgets.Select(
    options=source_entity_list,
    value=SOURCE_ENTITY_ID,
)
source_entity_picker

In [ ]:
#@markdown Verifying entity token of source...
if source_entity_picker.value != SOURCE_ENTITY_ID:
  # Switch entity
  response = requests.get(BASE_URL+"/refresh/"+source_entity_picker.value, headers=source_auth_headers)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  SOURCE_ENTITY_ID = source_entity_picker.value
print("Token verified")

In [ ]:
#@markdown Select source template(s) to copy:<br>
#@markdown _Hold down __Command__ to select multiple vales_<br>
payload = {"operationName":"LoadResources","variables":{"directory":"/","resourceType":"Template","searchTerms":[],"limit":100,"offset":0,"sortBy":"UpdatedAt","sortOrder":"DESC"},"query":"query LoadResources($directory: String, $resourceType: ResourceType, $searchTerms: [String!]!, $limit: Int!, $offset: Int!, $sortBy: ResourceSortAttributes!, $sortOrder: String!) {\n  resources(directory: $directory, resourceType: $resourceType, searchTerms: $searchTerms, limit: $limit, offset: $offset, sortBy: $sortBy, sortOrder: $sortOrder) {\n    resources {\n      ...resourceListFragment\n      statistics {\n        ...resourceListStatisticsFragment\n        __typename\n      }\n      __typename\n    }\n    totalCount\n    __typename\n  }\n}\n\nfragment resourceListFragment on Resource {\n  id\n  contentId\n  directory\n  resourceType\n  title\n  updatedAt\n  __typename\n}\n\nfragment resourceListStatisticsFragment on ResourceStatistics {\n  usedCount\n  __typename\n}\n"}
result = requests.post(INT_BASE_URL+"/graphql?op=LoadResources", json=payload, headers=source_auth_headers)
template_result = result.json()
if template_result.get("errors"):
  print(payload)
  print(template_result)
  raise TypeError("Could not find any templates!")
else:
  source_template_list = list(map(lambda x: (x["title"],x["contentId"]), template_result["data"]["resources"]["resources"]))
source_template_picker = widgets.SelectMultiple(
    options=source_template_list,
    value=[]
)
source_template_picker

In [ ]:
#@markdown Select destination entities to copy template to:<br>
#@markdown _Hold down __Command__ to select multiple vales_
destination_entity_list = list(map(lambda x: (x["name"],x["id"]), list(filter(lambda x: x["id"] != SOURCE_ENTITY_ID, user_data["entities"]))))
destination_picker = widgets.SelectMultiple(
    options=sorted(destination_entity_list),
    rows=10
)
destination_picker

In [ ]:
#@markdown Get auth tokens for destination entities...
if len(destination_picker.value) > 0:
  if last_header == "":
    last_header = source_auth_headers
  destination_entities = []
  for i,entity_id in enumerate(destination_picker.value):
    response = requests.get(BASE_URL+"/refresh/"+entity_id, headers=last_header)
    data = response.json()
    if "access_token" in data:
      last_header = { "Authorization": "bearer " + data["access_token"]}
      destination_entities.append({
        "id": entity_id,
        "name": destination_picker.label[i],
        "header": last_header
      })
      print("Authenticated to destination entity \"%s\"" % destination_picker.label[i])
    else:
      raise TypeError(data["errors"], "Could not authenicate to entity " + destination_picker.label[i])
else:
  raise TypeError("No entities selected")

In [ ]:
#@markdown Preparing template functions
def get_source_template(template_id):
  # Get Source template document items...
  payload = {"operationName":"Document","variables":{"documentId":template_id},"query":"query Document($documentId: String!) {\n  document(documentId: $documentId) {\n    ...documentFragment\n    __typename\n  }\n}\n\nfragment documentFragment on Document {\n  id\n  name\n  status\n  isSigning\n  isSigningOrder\n  isSelfsign\n  type\n  autoCommentText\n  autoCommentEmail\n  isAutoComment\n  isAutoCommentEmail\n  value\n  contractStartDate\n  contractEndDate\n  expirationDate\n  sendDate\n  signDate\n  createdAt\n  scheduledSendingTime\n  scheduledSendingDate\n  isSigningBiometric\n  isSigningInitials\n  isSigningForward\n  isPrivate\n  previewUrl\n  externalEditorId\n  externalEditorType\n  uniqueId\n  hasWriteAccess\n  isSmsReminder\n  isReminderSending\n  isIdentifyRecipient\n  reminderOpened\n  reminderOpenedDays\n  reminderSigned\n  reminderSignedDays\n  reminderRepeat\n  reminderRepeatDays\n  reminderExpiration\n  reminderExpirationDays\n  isSmsSending\n  isScheduledSending\n  tags {\n    ...tagFragment\n    __typename\n  }\n  user {\n    ...userFragment\n    __typename\n  }\n  recipients {\n    ...recipientFragment\n    __typename\n  }\n  video {\n    ...documentVideoFragment\n    __typename\n  }\n  pages {\n    ...pageFragment\n    __typename\n  }\n  manuallySignedPages {\n    ...manuallySignedPageFragment\n    __typename\n  }\n  documentAttachments {\n    ...documentAttachmentFragment\n    __typename\n  }\n  documentRevisions {\n    ...documentRevisionsFragment\n    __typename\n  }\n  __typename\n}\n\nfragment documentAttachmentFragment on DocumentAttachment {\n  id\n  restrictAccess\n  requireView\n  previewUrl\n  lastView\n  lastUpload\n  createdAt\n  attachment {\n    ...attachmentFragment\n    __typename\n  }\n  accessRecipientList {\n    ...accessRecipientFragment\n    __typename\n  }\n  __typename\n}\n\nfragment attachmentFragment on Attachment {\n  type\n  title\n  description\n  filename\n  size\n  sha\n  published\n  url\n  library\n  id\n  __typename\n}\n\nfragment accessRecipientFragment on Recipient {\n  id\n  firstName\n  lastName\n  fullName\n  roleName\n  __typename\n}\n\nfragment tagFragment on Tag {\n  title\n  status\n  id\n  __typename\n}\n\nfragment userFragment on User {\n  email\n  firstName\n  lastName\n  fullName\n  id\n  thumbUrl\n  title\n  mobile\n  signature\n  signatureBase30\n  __typename\n}\n\nfragment recipientFragment on Recipient {\n  firstName\n  lastName\n  fullName\n  companyName\n  companyNumber\n  title\n  mobile\n  email\n  thumbUrl\n  gender\n  role\n  roleName\n  id\n  orderNum\n  verifyQna\n  verifyQnaOpen\n  verifyQnaSign\n  verifyQnaQuestion\n  verifyQnaAnswer\n  verifySms\n  verifySmsOpen\n  verifySmsSign\n  verifyLinkedin\n  verifyEid\n  verifyEidType\n  createdAt\n  phone\n  note\n  status\n  contactId\n  __typename\n}\n\nfragment documentVideoFragment on DocumentVideo {\n  video {\n    id\n    thumbUrl\n    videoUrl\n    videoType\n    __typename\n  }\n  __typename\n}\n\nfragment manuallySignedPageFragment on SignaturePage {\n  pageWidth\n  pageHeight\n  pageNum\n  thumbUrl\n  __typename\n}\n\nfragment documentRevisionsFragment on DocumentRevision {\n  message\n  revision\n  status\n  editorBlockContentVersions {\n    blockId\n    contentId\n    versions {\n      versionId\n      lastModified\n      isLatest\n      isPublished\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment pageFragment on DocumentPage {\n  id\n  pageType\n  pageNum\n  pageHeight\n  pageWidth\n  cropBoxH\n  cropBoxW\n  video {\n    id\n    thumbUrl\n    videoUrl\n    videoType\n    videoTitle\n    __typename\n  }\n  contentLinkBlock {\n    buttonText\n    description\n    id\n    thumbUrl\n    thumbExtension\n    title\n    url\n    __typename\n  }\n  editorBlock {\n    ...editorBlockFragment\n    __typename\n  }\n  __typename\n}\n\nfragment editorBlockFragment on EditorBlock {\n  id\n  entityId\n  userId\n  contentId\n  pdfPageCount\n  content {\n    ...editorBlockContentFragment\n    __typename\n  }\n  __typename\n}\n\nfragment editorBlockContentFragment on EditorContent {\n  id\n  displayName\n  fontSettings {\n    defaultFont {\n      ...fontFragment\n      __typename\n    }\n    headingsFont {\n      ...fontFragment\n      __typename\n    }\n    __typename\n  }\n  sections {\n    id\n    displayName\n    rows {\n      id\n      type\n      columnCount\n      cells {\n        id\n        width\n        type\n        nodes {\n          id\n          type\n          content\n          srcUrl\n          imageId\n          linkUrl\n          fields {\n            id\n            type\n            category\n            mergeKey\n            recipientId\n            userId\n            customName\n            value\n            inputSettings {\n              type\n              required\n              label\n              richTextLabel\n              helpText\n              connectedId\n              width\n              minValue\n              maxValue\n              placeholder\n              options\n              __typename\n            }\n            __typename\n          }\n          inputFieldSets {\n            ids\n            __typename\n          }\n          caption\n          heading\n          imageSize\n          imageAlign\n          align\n          url\n          videoId\n          imageTransforms {\n            translateX\n            translateY\n            scale\n            __typename\n          }\n          dividerType\n          dividerPadding\n          tableStyle {\n            borderColor\n            headerRowBackgroundColor\n            alternateRowColor\n            __typename\n          }\n          name\n          preCalculated\n          locked\n          pricingTableSections {\n            id\n            displayName\n            columns {\n              id\n              name\n              displayName\n              width\n              enabled\n              __typename\n            }\n            rows {\n              id\n              values {\n                columnId\n                value\n                __typename\n              }\n              discountFlatFee\n              taxFlatFee\n              optionalProductFieldId\n              variableQuantityFieldId\n              __typename\n            }\n            sectionSummary {\n              discount {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              tax {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              price {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              __typename\n            }\n            __typename\n          }\n          pricingTableSummaryValues {\n            displayName\n            discount {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            tax {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            price {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            __typename\n          }\n          currencySettings {\n            formatOptions {\n              currency\n              currencyDisplay\n              __typename\n            }\n            locale\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment fontFragment on UpsertFont {\n  fontFamily\n  displayName\n  sources {\n    fontStyle\n    fontWeight\n    unicodeRange\n    format\n    src\n    __typename\n  }\n  __typename\n}\n"}
  response = requests.post("https://int.getaccept.com/graphql?op=Document", json=payload, headers=source_auth_headers)
  source_template = response.json()
  source_pages = source_template["data"]["document"]["pages"]
  source_recipients = source_template["data"]["document"]["recipients"]
  source_attachments = source_template["data"]["document"]["documentAttachments"]
  source_template_name = source_template["data"]["document"]["name"]
  print("Found {} page(s), {} role(s), {} attachment(s) ".format(len(list(source_pages)),len(list(source_recipients)),len(list(source_attachments))))
  return source_pages, source_recipients, source_attachments, source_template_name

def sync_recipient_roles(source_recipients, destination_template_id, destination_recipients, destination_auth_headers):
  # Sync recipients/roles
  if destination_recipients != source_recipients:
    # Get dummy contact (temporary solution)
    payload = {"operationName":"contacts","variables":{"payload":{"status":"Active","orderBy":"firstName:ASC","offset":0,"limit":1}},"query":"query contacts($payload: GetContactsInput!) {\n  contacts(payload: $payload) {\n    id\n    createdAt\n    firstName\n    lastName\n    title\n    mobile\n    email\n    thumbUrl\n    note\n    companyName\n    companyNumber\n    documentCount\n    __typename\n  }\n}\n"}
    response = requests.post(INT_BASE_URL+"/graphql?op=contacts", json=payload, headers=destination_auth_headers)
    contact_result = response.json()
    if contact_result.get("errors"):
      print(contact_result)
    else:
      contact_id = contact_result["data"]["contacts"][0]["id"]

    # Remove existing recipients
    for recipient in destination_recipients:
      payload = {"operationName":"DeleteRecipient","variables":{"id":recipient["id"]},"query":"mutation DeleteRecipient($id: String!) {\n  deleteRecipient(id: $id)\n}\n"}
      response = requests.post("https://int.getaccept.com/graphql?op=DeleteRecipient", json=payload, headers=destination_auth_headers)
    
    # Add from source recipients
    for recipient in source_recipients:
      # Add new recipient from contact
      payload = {"operationName":"CreateRecipient","variables":{"documentId":destination_template_id, "contactId": contact_id, "role": "signer" },"query":"mutation CreateRecipient($documentId: String!, $contactId: String!, $role: RoleType) {\n  createRecipient(documentId: $documentId, contactId: $contactId, role: $role) {\n    id\n    __typename\n  }\n}\n"}
      result = requests.post("https://int.getaccept.com/graphql?op=CreateRecipient", json=payload, headers=destination_auth_headers)
      create_recipient_result = result.json()
      if create_recipient_result.get("errors"):
        print(create_recipient_result)
      else:
        recipient_id = create_recipient_result["data"]["createRecipient"]["id"]
        # Update added recipient
        verifyEidType = recipient["verifyEidType"] if recipient["verifyEidType"] else "bankid"
        payload = {"operationName":"UpdateRecipientV2","variables":{"id":recipient_id,"documentId":destination_template_id,"firstName":"","lastName":"","fullName":"","companyName":"","companyNumber":"","title":"","mobile":"","email":"","thumbUrl":"","gender":"","role":recipient["role"],"roleName":recipient["roleName"],"orderNum":recipient["orderNum"],"verifyQna":bool(recipient["verifyQna"]),"verifyQnaOpen":bool(recipient["verifyQnaOpen"]),"verifyQnaSign":bool(recipient["verifyQnaOpen"]),"verifyQnaQuestion":"","verifyQnaAnswer":"","verifySms":bool(recipient["verifySms"]),"verifySmsOpen":bool(recipient["verifySmsOpen"]),"verifySmsSign":bool(recipient["verifySmsSign"]),"verifyLinkedin":bool(recipient["verifyLinkedin"]),"verifyEid":bool(recipient["verifyEid"]),"createdAt":recipient["createdAt"],"phone":"","note":"","status":"added","contactId":"","__typename":"Recipient","verifyEidType":verifyEidType},"query":"mutation UpdateRecipientV2($documentId: String!, $id: String!, $contactId: String, $role: String, $roleName: String, $status: String, $orderNum: Int, $firstName: String, $lastName: String, $companyId: String, $companyName: String, $companyNumber: String, $title: String, $phone: String, $mobile: String, $email: String, $thumbUrl: String, $note: String, $gender: String, $verifyQna: Boolean, $verifyQnaOpen: Boolean, $verifyQnaSign: Boolean, $verifyQnaQuestion: String, $verifyQnaAnswer: String, $verifySms: Boolean, $verifySmsOpen: Boolean, $verifySmsSign: Boolean, $verifySmsNumber: String, $verifySmsCode: String, $verifyLinkedin: Boolean, $verifyEid: Boolean, $verifyEidType: EIdSignType, $verifyEidNumber: String) {\n  updateRecipientV2(documentId: $documentId, id: $id, contactId: $contactId, role: $role, roleName: $roleName, status: $status, orderNum: $orderNum, firstName: $firstName, lastName: $lastName, companyId: $companyId, companyName: $companyName, companyNumber: $companyNumber, title: $title, phone: $phone, mobile: $mobile, email: $email, thumbUrl: $thumbUrl, note: $note, gender: $gender, verifyQna: $verifyQna, verifyQnaOpen: $verifyQnaOpen, verifyQnaSign: $verifyQnaSign, verifyQnaQuestion: $verifyQnaQuestion, verifyQnaAnswer: $verifyQnaAnswer, verifySms: $verifySms, verifySmsOpen: $verifySmsOpen, verifySmsSign: $verifySmsSign, verifySmsNumber: $verifySmsNumber, verifySmsCode: $verifySmsCode, verifyLinkedin: $verifyLinkedin, verifyEid: $verifyEid, verifyEidType: $verifyEidType, verifyEidNumber: $verifyEidNumber)\n}\n"}
        result = requests.post("https://int.getaccept.com/graphql?op=UpdateRecipientV2", json=payload, headers=destination_auth_headers)
        update_recipient_result = result.json()
        if update_recipient_result.get("errors"):
          print(update_recipient_result)
        else:
          destination_recipients.append({
            "source_recipient_id": recipient["id"],
            "destination_recipient_id": recipient_id
          })
          print("- Added role {}".format(recipient["roleName"]))
    return destination_recipients

def sync_attachments(source_attachments, destination_template_id, destination_attachments, destination_auth_headers):
  # Check and sync source and destination attachments
  filtered_source_attachments = []
  filtered_destination_attachments = []
  for i,att in enumerate(source_attachments):
    if att["attachment"]["type"] == "external" or att["attachment"]["type"] == "file":
      filtered_source_attachments.append( [i,att["attachment"]["title"],att["attachment"]["url"]] )
  for i,att in enumerate(destination_attachments):
    if att["attachment"]["type"] == "external" or att["attachment"]["type"] == "file":
      filtered_destination_attachments.append( [i,att["attachment"]["title"],att["attachment"]["url"]] )
  if filtered_source_attachments != filtered_destination_attachments:
    # Delete existing attachments for destination
    delete_destination_attachment(destination_template_id, destination_attachments, destination_auth_headers)

    for index,att in enumerate(source_attachments):
      # Create new attachment reference
      payload = {"operationName":"CreateAttachment","variables":{"description":att["attachment"]["description"],"published":0,"library":0,"title":att["attachment"]["title"],"url":att["attachment"]["url"],"type":att["attachment"]["type"]},"query":"mutation CreateAttachment($type: AttachmentType!, $file: Upload, $url: String, $title: String, $description: String = \"\", $published: Float = 0, $library: Float = 0) {\n  createAttachment(type: $type, file: $file, url: $url, title: $title, description: $description, published: $published, library: $library) {\n    ...attachmentFragment\n    __typename\n  }\n}\n\nfragment attachmentFragment on Attachment {\n  type\n  title\n  description\n  filename\n  size\n  sha\n  published\n  url\n  library\n  id\n  __typename\n}\n"}
      if att["attachment"]["type"] == "file":
        # Download attachment file
        file_url = att["attachment"]["url"]
        response = requests.get(file_url, timeout=10)
        file_data = response.content
        file_name = file_url.split("/")[-1:][0].split("?")[0]
        file_ext = file_name.split(".")[-1:][0]
        file_content_type = mimetypes.guess_type(file_name)[0]
        files = [
          ('1', (file_name, file_data, file_content_type))
        ]
        payload["variables"]["url"] = ""
        data = {
            "operations": json.dumps(payload),
            "map": json.dumps({ "1": ["variables.file"] })
        }
        result = requests.request("POST", "https://int.getaccept.com/graphql?op=CreateAttachment", files=files, data=data, headers=destination_auth_headers)
      else:
        result = requests.request("POST", "https://int.getaccept.com/graphql?op=CreateAttachment", json=payload, headers=destination_auth_headers)
      create_attachment_result = result.json()
      if create_attachment_result.get("errors"):
        print(create_attachment_result)
        print(data)
      elif not "data" in create_attachment_result:
        print("Error uploading attachment")
        print(result)
      else:
        attachment_id = create_attachment_result["data"]["createAttachment"]["id"]
        # Add attachment to document
        payload = {"operationName":"AddAttachmentToDocument","variables":{"accessRecipientList":[],"documentId":destination_template_id,"attachmentId":attachment_id,"requireView":0,"restrictAccess":0},"query":"mutation AddAttachmentToDocument($documentId: String!, $attachmentId: String!, $restrictAccess: Int, $requireView: Int, $accessRecipientList: [String!] = []) {\n  addDocumentAttachment(documentId: $documentId, attachmentId: $attachmentId, restrictAccess: $restrictAccess, requireView: $requireView, accessRecipientList: $accessRecipientList) {\n    ...documentAttachmentFragment\n    __typename\n  }\n}\n\nfragment documentAttachmentFragment on DocumentAttachment {\n  id\n  restrictAccess\n  requireView\n  previewUrl\n  lastView\n  lastUpload\n  createdAt\n  attachment {\n    ...attachmentFragment\n    __typename\n  }\n  accessRecipientList {\n    ...accessRecipientFragment\n    __typename\n  }\n  __typename\n}\n\nfragment attachmentFragment on Attachment {\n  type\n  title\n  description\n  filename\n  size\n  sha\n  published\n  url\n  library\n  id\n  __typename\n}\n\nfragment accessRecipientFragment on Recipient {\n  id\n  firstName\n  lastName\n  fullName\n  roleName\n  __typename\n}\n"}
        result = requests.post("https://int.getaccept.com/graphql?op=AddAttachmentToDocument", json=payload, headers=destination_auth_headers)
        update_recipient_result = result.json()
        if update_recipient_result.get("errors"):
          print(update_recipient_result)
        else:
          print("- Added attachment {}".format(att["id"]))

def delete_destination_attachment(destination_template_id, destination_attachments, destination_auth_headers):
  # Delete existing destination attachment
  for index,att in enumerate(destination_attachments):
    payload = {"operationName":"RemoveDocumentAttachment","variables":{"id":att["id"]},"query":"mutation RemoveDocumentAttachment($id: String!) {\n  removeDocumentAttachment(id: $id)\n}\n"}
    result = requests.post("https://int.getaccept.com/graphql?op=RemoveDocumentAttachment", json=payload, headers=destination_auth_headers)
    delete_result = result.json()
    if delete_result.get("errors"):
      print(delete_result)
    else:
      print("- Attachment {} deleted".format(att["id"]))

def get_destination_template_data(destination_template_id,destination_auth_headers):
  # Get Destination template data
  payload = {"operationName":"Document","variables":{"documentId":destination_template_id},"query":"query Document($documentId: String!) {\n  document(documentId: $documentId) {\n    ...documentFragment\n    __typename\n  }\n}\n\nfragment documentFragment on Document {\n  id\n  name\n  status\n  isSigning\n  isSigningOrder\n  isSelfsign\n  type\n  autoCommentText\n  autoCommentEmail\n  isAutoComment\n  isAutoCommentEmail\n  value\n  contractStartDate\n  contractEndDate\n  expirationDate\n  sendDate\n  signDate\n  createdAt\n  scheduledSendingTime\n  scheduledSendingDate\n  isSigningBiometric\n  isSigningInitials\n  isSigningForward\n  isPrivate\n  previewUrl\n  externalEditorId\n  externalEditorType\n  uniqueId\n  hasWriteAccess\n  isSmsReminder\n  isReminderSending\n  isIdentifyRecipient\n  reminderOpened\n  reminderOpenedDays\n  reminderSigned\n  reminderSignedDays\n  reminderRepeat\n  reminderRepeatDays\n  reminderExpiration\n  reminderExpirationDays\n  isSmsSending\n  isScheduledSending\n  tags {\n    ...tagFragment\n    __typename\n  }\n  user {\n    ...userFragment\n    __typename\n  }\n  recipients {\n    ...recipientFragment\n    __typename\n  }\n  video {\n    ...documentVideoFragment\n    __typename\n  }\n  pages {\n    ...pageFragment\n    __typename\n  }\n  manuallySignedPages {\n    ...manuallySignedPageFragment\n    __typename\n  }\n  documentAttachments {\n    ...documentAttachmentFragment\n    __typename\n  }\n  documentRevisions {\n    ...documentRevisionsFragment\n    __typename\n  }\n  __typename\n}\n\nfragment documentAttachmentFragment on DocumentAttachment {\n  id\n  restrictAccess\n  requireView\n  previewUrl\n  lastView\n  lastUpload\n  createdAt\n  attachment {\n    ...attachmentFragment\n    __typename\n  }\n  accessRecipientList {\n    ...accessRecipientFragment\n    __typename\n  }\n  __typename\n}\n\nfragment attachmentFragment on Attachment {\n  type\n  title\n  description\n  filename\n  size\n  sha\n  published\n  url\n  library\n  id\n  __typename\n}\n\nfragment accessRecipientFragment on Recipient {\n  id\n  firstName\n  lastName\n  fullName\n  roleName\n  __typename\n}\n\nfragment tagFragment on Tag {\n  title\n  status\n  id\n  __typename\n}\n\nfragment userFragment on User {\n  email\n  firstName\n  lastName\n  fullName\n  id\n  thumbUrl\n  title\n  mobile\n  signature\n  signatureBase30\n  __typename\n}\n\nfragment recipientFragment on Recipient {\n  firstName\n  lastName\n  fullName\n  companyName\n  companyNumber\n  title\n  mobile\n  email\n  thumbUrl\n  gender\n  role\n  roleName\n  id\n  orderNum\n  verifyQna\n  verifyQnaOpen\n  verifyQnaSign\n  verifyQnaQuestion\n  verifyQnaAnswer\n  verifySms\n  verifySmsOpen\n  verifySmsSign\n  verifyLinkedin\n  verifyEid\n  verifyEidType\n  createdAt\n  phone\n  note\n  status\n  contactId\n  __typename\n}\n\nfragment documentVideoFragment on DocumentVideo {\n  video {\n    id\n    thumbUrl\n    videoUrl\n    videoType\n    __typename\n  }\n  __typename\n}\n\nfragment manuallySignedPageFragment on SignaturePage {\n  pageWidth\n  pageHeight\n  pageNum\n  thumbUrl\n  __typename\n}\n\nfragment documentRevisionsFragment on DocumentRevision {\n  message\n  revision\n  status\n  editorBlockContentVersions {\n    blockId\n    contentId\n    versions {\n      versionId\n      lastModified\n      isLatest\n      isPublished\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment pageFragment on DocumentPage {\n  id\n  pageType\n  pageNum\n  pageHeight\n  pageWidth\n  cropBoxH\n  cropBoxW\n  video {\n    id\n    thumbUrl\n    videoUrl\n    videoType\n    videoTitle\n    __typename\n  }\n  contentLinkBlock {\n    buttonText\n    description\n    id\n    thumbUrl\n    thumbExtension\n    title\n    url\n    __typename\n  }\n  editorBlock {\n    ...editorBlockFragment\n    __typename\n  }\n  __typename\n}\n\nfragment editorBlockFragment on EditorBlock {\n  id\n  entityId\n  userId\n  contentId\n  pdfPageCount\n  content {\n    ...editorBlockContentFragment\n    __typename\n  }\n  __typename\n}\n\nfragment editorBlockContentFragment on EditorContent {\n  id\n  displayName\n  fontSettings {\n    defaultFont {\n      ...fontFragment\n      __typename\n    }\n    headingsFont {\n      ...fontFragment\n      __typename\n    }\n    __typename\n  }\n  sections {\n    id\n    displayName\n    rows {\n      id\n      type\n      columnCount\n      cells {\n        id\n        width\n        type\n        nodes {\n          id\n          type\n          content\n          srcUrl\n          imageId\n          linkUrl\n          fields {\n            id\n            type\n            category\n            mergeKey\n            recipientId\n            userId\n            customName\n            value\n            inputSettings {\n              type\n              required\n              label\n              richTextLabel\n              helpText\n              connectedId\n              width\n              minValue\n              maxValue\n              placeholder\n              options\n              __typename\n            }\n            __typename\n          }\n          inputFieldSets {\n            ids\n            __typename\n          }\n          caption\n          heading\n          imageSize\n          imageAlign\n          align\n          url\n          videoId\n          imageTransforms {\n            translateX\n            translateY\n            scale\n            __typename\n          }\n          dividerType\n          dividerPadding\n          tableStyle {\n            borderColor\n            headerRowBackgroundColor\n            alternateRowColor\n            __typename\n          }\n          name\n          preCalculated\n          locked\n          pricingTableSections {\n            id\n            displayName\n            columns {\n              id\n              name\n              displayName\n              width\n              enabled\n              __typename\n            }\n            rows {\n              id\n              values {\n                columnId\n                value\n                __typename\n              }\n              discountFlatFee\n              taxFlatFee\n              optionalProductFieldId\n              variableQuantityFieldId\n              __typename\n            }\n            sectionSummary {\n              discount {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              tax {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              price {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              __typename\n            }\n            __typename\n          }\n          pricingTableSummaryValues {\n            displayName\n            discount {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            tax {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            price {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            __typename\n          }\n          currencySettings {\n            formatOptions {\n              currency\n              currencyDisplay\n              __typename\n            }\n            locale\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment fontFragment on UpsertFont {\n  fontFamily\n  displayName\n  sources {\n    fontStyle\n    fontWeight\n    unicodeRange\n    format\n    src\n    __typename\n  }\n  __typename\n}\n"}
  response = requests.post("https://int.getaccept.com/graphql?op=Document", json=payload, headers=destination_auth_headers)
  destination_template = response.json()
  destination_pages = destination_template["data"]["document"]["pages"]
  destination_recipients = destination_template["data"]["document"]["recipients"]
  destination_attachments = destination_template["data"]["document"]["documentAttachments"]
  return destination_pages, destination_recipients, destination_attachments

def delete_destination_pages(destination_template_id, destination_pages, destination_auth_headers):
  # Delete existing destination pages
  for index,page in enumerate(destination_pages):
    payload = {"operationName":"DeleteDocumentPage","variables":{"documentId":destination_template_id,"documentPageId":page["id"]},"query":"mutation DeleteDocumentPage($documentId: String!, $documentPageId: String!) {\n  deleteDocumentPage(documentId: $documentId, documentPageId: $documentPageId)\n}\n"}
    result = requests.post("https://int.getaccept.com/graphql?op=DeleteDocumentPage", json=payload, headers=destination_auth_headers)
    delete_result = result.json()
    if delete_result.get("errors"):
      print(delete_result)
    else:
      print("- Page {} deleted".format(page["pageNum"]))

# Define content block functions
def get_content_rows(section_index, page_index, source_pages, destination_recipients, destination_auth_headers):
  # Get content rows for a section
  editor_pages = copy.deepcopy(source_pages)
  editor_rows = editor_pages[page_index]["editorBlock"]["content"]["sections"][section_index]["rows"]
  for index,row in enumerate(editor_rows):
    row["rowId"] = row["id"]
    del row["id"]
    row["rowType"] = row["type"]
    del row["type"]
    del row["__typename"]
    for index,cell in enumerate(row.get("cells")):
      cell["cellId"] = cell["id"]
      del cell["id"]
      cell["cellType"] = cell["type"]
      del cell["type"]
      del cell["__typename"]
      for index,node in enumerate(cell.get("nodes")):
        if node["type"] == "Image":
          if node.get("srcUrl"):
            if (node["srcUrl"].find("amazonaws.com")) != -1:
              new_image = copy_image_file(node["srcUrl"], destination_auth_headers)
            else:
              new_image = copy_image_url(node["srcUrl"], destination_auth_headers)
          else:
            new_image = copy_image_id(node["imageId"], destination_auth_headers)            
          node["imageId"] = new_image["id"]
          node["url"] = new_image["url"]
        node["nodeId"] = node["id"]
        del node["id"]
        node["nodeType"] = node["type"]
        del node["type"]
        del node["__typename"]
        del node["srcUrl"]
        if node.get("tableStyle"):
          del node["tableStyle"]["__typename"]
        if node.get("imageTransforms"):
          del node["imageTransforms"]["__typename"]
        if node.get("fields"):
          for index,field in enumerate(node.get("fields")):
            del field["__typename"]
            if field.get("inputSettings"):
              del field["inputSettings"]["__typename"]
            if field.get("recipientId"):
              field["recipientId"] = replace_recipient(field["recipientId"], destination_recipients)
        if node.get("inputFieldSets"):
          for index,fieldset in enumerate(node.get("inputFieldSets")):
            del fieldset["__typename"]
  return editor_rows

def copy_image_url(image_url, destination_auth_headers):
  # Copy new image from url
  payload = {"operationName":"CreateImageV2","variables":{"url":image_url,"type":"external"},"query":"mutation CreateImageV2($type: ImageType!, $fileInfo: FileInfo, $url: String, $name: String) {\n  createImageV2(type: $type, fileInfo: $fileInfo, url: $url, name: $name) {\n    image {\n      id\n      url\n      imageUrl\n      __typename\n    }\n    uploadLink {\n      url\n      headers\n      __typename\n    }\n    __typename\n  }\n}\n"}
  response = requests.post(INT_BASE_URL+"/graphql?op=CreateImageV2", json=payload, headers=destination_auth_headers)
  image_result = response.json()
  if image_result.get("errors"):
    print(payload)
    print(image_result)
  else:
    return { "id": image_result["data"]["createImageV2"]["image"]["id"], "url": "" }

def copy_image_file(image_url, destination_auth_headers):
  # Copy uploaded source image to destination
  file_name = image_url.split("/")[-1:][0].split("?")[0]
  file_ext = file_name.split(".")[-1:][0]
  file_content_type = mimetypes.guess_type(file_name)[0]
  response = requests.get(image_url, timeout=10)
  file_data = response.content
  source_image = Image.open(io.BytesIO(file_data))

  # Create image upload url
  payload = {"operationName":"CreateImageV2","variables":{"type":"file","fileInfo":{"filename":file_name,"width":source_image.width,"height":source_image.height}},"query":"mutation CreateImageV2($type: ImageType!, $fileInfo: FileInfo, $url: String, $name: String) {\n  createImageV2(type: $type, fileInfo: $fileInfo, url: $url, name: $name) {\n    image {\n      id\n      url\n      imageUrl\n      __typename\n    }\n    uploadLink {\n      url\n      headers\n      __typename\n    }\n    __typename\n  }\n}\n"}
  result = requests.post(INT_BASE_URL+"/graphql?op=CreateImageV2", json=payload, headers=destination_auth_headers)
  create_image_result = result.json()
  if create_image_result.get("errors"):
    print(payload)
    print(create_image_result)
    raise ValueError(create_image_result["errors"][0]["message"])
  else:
    new_image_id = create_image_result["data"]["createImageV2"]["image"]["id"]
    new_image_url = create_image_result["data"]["createImageV2"]["image"]["url"]
    presigned_url = create_image_result["data"]["createImageV2"]["uploadLink"]["url"]
    presigned_headers = dict(item.split(":") for item in create_image_result["data"]["createImageV2"]["uploadLink"]["headers"].split(","))
    # Upload file
    files=[
      ('file', ('file', file_data))
    ]
    presigned_headers["Content-Type"] = file_content_type
    result = requests.request("POST", presigned_url, data=presigned_headers, files=files)
    if result.status_code == 204:
      source_image.close()
      return { "id": new_image_id, "url": "" }
    else:
      print("Error uploading file: " + result.status_code)
      print(result.text)

def copy_image_id(image_id, destination_auth_headers):
  # Get Source image url
  payload = {"operationName":"image","variables":{"id":image_id,"height":"1440"},"query":"query image($id: String!, $height: String) {\n  image(id: $id, height: $height) {\n    url\n    __typename\n  }\n}\n"}
  response = requests.post("https://int.getaccept.com/graphql?op=image", json=payload, headers=source_auth_headers)
  image_result = response.json()
  if image_result.get("errors"):
    print(payload)
    print(image_result)
    raise ValueError(image_result["errors"][0]["message"])
  else:
    image_url = image_result["data"]["image"]["url"]
    if (image_url.find("amazonaws.com")) != -1:
      return copy_image_file(image_url, destination_auth_headers)
    else:
      return copy_image_url(image_url, destination_auth_headers)

def add_document_page(payload, destination_auth_headers):
  # Add document page
  response = requests.post(INT_BASE_URL+"/graphql?op=AddDocumentPage", json=payload, headers=destination_auth_headers)
  page_result = response.json()
  if page_result.get("errors"):
    print(payload)
    print(page_result)
    raise ValueError(page_result["errors"][0]["message"])
  else:
    print("- Page {} copied".format(payload["variables"]["pageNum"]))

def upload_link_block(block_id, image_url, destination_auth_headers):
  # Upload existing link thumb to entity
  payload = {"operationName":"ContentLinkBlockUpload","variables":{"blockId":block_id},"query":"query ContentLinkBlockUpload($blockId: String!) {\n  contentLinkBlockUpload(id: $blockId)\n}\n"}
  response = requests.post(INT_BASE_URL+"/graphql?op=ContentLinkBlockUpload", json=payload, headers=destination_auth_headers)
  page_result = response.json()
  if page_result.get("errors"):
    print(payload)
    print(page_result)
    raise ValueError(page_result["errors"][0]["message"])
  else:
    upload_block_url = page_result["data"]["contentLinkBlockUpload"]
    # Download image
    response = requests.get(image_url, timeout=10)
    file_data = response.content
    # Upload file
    result = requests.request("PUT", upload_block_url, data=file_data)
    if result.status_code == 200:
      return True
    else:
      print("Error uploading link block image: " + result.status_code)
      print(result.text)
  
def replace_recipient(recipient_id, recipients):
  for rec in recipients:
    if rec.get("source_recipient_id") and rec["source_recipient_id"] == recipient_id:
      return rec["destination_recipient_id"]
  # not found
  return ""

def copy_pages(source_pages, destination_template_id, destination_recipients, destination_auth_headers):
  # Copy pages from source template pages
  page_types = ["Unknown","","DocumentContent","contentLinkBlock","video","editorBlock"]
  for index,page in enumerate(source_pages):
    page_type = page_types[page["pageType"]]
    if page["pageType"] == 3:
      # Add link block
      block_data = page["contentLinkBlock"]
      block_data["__typename"] = "ContentLinkPreview"
      block_data["id"] = ""
      block_data["documentId"] = destination_template_id
      block_data["user"] = None
      thumb_url = block_data["thumbUrl"]
      block_data["thumbUrl"] = None
      payload = {"operationName":"CreateContentLinkBlock","variables":block_data,"query":"mutation CreateContentLinkBlock($url: String!, $title: String!, $thumbUrl: String, $thumbExtension: String, $description: String!, $buttonText: String!) {\n  createContentLinkBlock(url: $url, title: $title, thumbUrl: $thumbUrl, thumbExtension: $thumbExtension, description: $description, buttonText: $buttonText) {\n    id\n    url\n    title\n    thumbUrl\n    thumbExtension\n    buttonText\n    description\n    __typename\n  }\n}\n"}
      result = requests.post(INT_BASE_URL+"/graphql?op=CreateContentLinkBlock", json=payload, headers=destination_auth_headers)
      block_result = result.json()
      if block_result.get("errors"):
        print(payload)
        print(block_result)
        raise ValueError(block_result["errors"][0]["message"])
      else:
        block_id = block_result["data"]["createContentLinkBlock"]["id"]
        
        # Upload link block image
        upload_link_block(block_id, thumb_url, destination_auth_headers)

        # Add page
        payload = {"operationName":"AddDocumentPage","variables":{"documentId":destination_template_id,"pageType":page_type,"pageNum":int(page["pageNum"]),"blockId":block_id},"query":"mutation AddDocumentPage($documentId: String!, $pageNum: Int!, $videoId: String, $blockId: String, $pageType: DocumentPageType, $defaultEditorRows: [CreateRowInput!], $fontSettings: InputFontSettings) {\n  addDocumentPage(documentPageInput: {documentId: $documentId, pageNum: $pageNum, videoId: $videoId, blockId: $blockId, pageType: $pageType, defaultEditorRows: $defaultEditorRows, fontSettings: $fontSettings}) {\n    id\n    thumbUrl\n    pageType\n    pageNum\n    video {\n      id\n      videoUrl\n      thumbUrl\n      __typename\n    }\n    contentLinkBlock {\n      buttonText\n      description\n      id\n      thumbUrl\n      thumbExtension\n      title\n      url\n      __typename\n    }\n    editorBlock {\n      ...editorBlockFragment\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment editorBlockFragment on EditorBlock {\n  id\n  entityId\n  userId\n  contentId\n  pdfPageCount\n  content {\n    ...editorBlockContentFragment\n    __typename\n  }\n  __typename\n}\n\nfragment editorBlockContentFragment on EditorContent {\n  id\n  displayName\n  fontSettings {\n    defaultFont {\n      ...fontFragment\n      __typename\n    }\n    headingsFont {\n      ...fontFragment\n      __typename\n    }\n    __typename\n  }\n  sections {\n    id\n    displayName\n    rows {\n      id\n      type\n      columnCount\n      cells {\n        id\n        width\n        type\n        nodes {\n          id\n          type\n          content\n          srcUrl\n          imageId\n          linkUrl\n          fields {\n            id\n            type\n            category\n            mergeKey\n            recipientId\n            userId\n            customName\n            value\n            inputSettings {\n              type\n              required\n              label\n              richTextLabel\n              helpText\n              connectedId\n              width\n              minValue\n              maxValue\n              placeholder\n              options\n              __typename\n            }\n            __typename\n          }\n          inputFieldSets {\n            ids\n            __typename\n          }\n          caption\n          heading\n          imageSize\n          imageAlign\n          align\n          url\n          videoId\n          imageTransforms {\n            translateX\n            translateY\n            scale\n            __typename\n          }\n          dividerType\n          dividerPadding\n          tableStyle {\n            borderColor\n            headerRowBackgroundColor\n            alternateRowColor\n            __typename\n          }\n          name\n          preCalculated\n          locked\n          pricingTableSections {\n            id\n            displayName\n            columns {\n              id\n              name\n              displayName\n              width\n              enabled\n              __typename\n            }\n            rows {\n              id\n              values {\n                columnId\n                value\n                __typename\n              }\n              discountFlatFee\n              taxFlatFee\n              optionalProductFieldId\n              variableQuantityFieldId\n              __typename\n            }\n            sectionSummary {\n              discount {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              tax {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              price {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              __typename\n            }\n            __typename\n          }\n          pricingTableSummaryValues {\n            displayName\n            discount {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            tax {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            price {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            __typename\n          }\n          currencySettings {\n            formatOptions {\n              currency\n              currencyDisplay\n              __typename\n            }\n            locale\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment fontFragment on UpsertFont {\n  fontFamily\n  displayName\n  sources {\n    fontStyle\n    fontWeight\n    unicodeRange\n    format\n    src\n    __typename\n  }\n  __typename\n}\n"}
        add_document_page(payload, destination_auth_headers)
        
    if page["pageType"] == 5:
      # Add content block
      content_rows = get_content_rows(0, index, source_pages, destination_recipients, destination_auth_headers)

      # Add page
      payload = {"operationName":"AddDocumentPage","variables":{"documentId":destination_template_id,"pageType":page_type,"pageNum":int(page["pageNum"]),"defaultEditorRows":content_rows},"query":"mutation AddDocumentPage($documentId: String!, $pageNum: Int!, $videoId: String, $blockId: String, $pageType: DocumentPageType, $defaultEditorRows: [CreateRowInput!], $fontSettings: InputFontSettings) {\n  addDocumentPage(documentPageInput: {documentId: $documentId, pageNum: $pageNum, videoId: $videoId, blockId: $blockId, pageType: $pageType, defaultEditorRows: $defaultEditorRows, fontSettings: $fontSettings}) {\n    id\n    thumbUrl\n    pageType\n    pageNum\n    video {\n      id\n      videoUrl\n      thumbUrl\n      __typename\n    }\n    contentLinkBlock {\n      buttonText\n      description\n      id\n      thumbUrl\n      thumbExtension\n      title\n      url\n      __typename\n    }\n    editorBlock {\n      ...editorBlockFragment\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment editorBlockFragment on EditorBlock {\n  id\n  entityId\n  userId\n  contentId\n  pdfPageCount\n  content {\n    ...editorBlockContentFragment\n    __typename\n  }\n  __typename\n}\n\nfragment editorBlockContentFragment on EditorContent {\n  id\n  displayName\n  fontSettings {\n    defaultFont {\n      ...fontFragment\n      __typename\n    }\n    headingsFont {\n      ...fontFragment\n      __typename\n    }\n    __typename\n  }\n  sections {\n    id\n    displayName\n    rows {\n      id\n      type\n      columnCount\n      cells {\n        id\n        width\n        type\n        nodes {\n          id\n          type\n          content\n          srcUrl\n          imageId\n          linkUrl\n          fields {\n            id\n            type\n            category\n            mergeKey\n            recipientId\n            userId\n            customName\n            value\n            inputSettings {\n              type\n              required\n              label\n              richTextLabel\n              helpText\n              connectedId\n              width\n              minValue\n              maxValue\n              placeholder\n              options\n              __typename\n            }\n            __typename\n          }\n          inputFieldSets {\n            ids\n            __typename\n          }\n          caption\n          heading\n          imageSize\n          imageAlign\n          align\n          url\n          videoId\n          imageTransforms {\n            translateX\n            translateY\n            scale\n            __typename\n          }\n          dividerType\n          dividerPadding\n          tableStyle {\n            borderColor\n            headerRowBackgroundColor\n            alternateRowColor\n            __typename\n          }\n          name\n          preCalculated\n          locked\n          pricingTableSections {\n            id\n            displayName\n            columns {\n              id\n              name\n              displayName\n              width\n              enabled\n              __typename\n            }\n            rows {\n              id\n              values {\n                columnId\n                value\n                __typename\n              }\n              discountFlatFee\n              taxFlatFee\n              optionalProductFieldId\n              variableQuantityFieldId\n              __typename\n            }\n            sectionSummary {\n              discount {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              tax {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              price {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              __typename\n            }\n            __typename\n          }\n          pricingTableSummaryValues {\n            displayName\n            discount {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            tax {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            price {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            __typename\n          }\n          currencySettings {\n            formatOptions {\n              currency\n              currencyDisplay\n              __typename\n            }\n            locale\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment fontFragment on UpsertFont {\n  fontFamily\n  displayName\n  sources {\n    fontStyle\n    fontWeight\n    unicodeRange\n    format\n    src\n    __typename\n  }\n  __typename\n}\n"}
      add_document_page(payload, destination_auth_headers)
    
    if page["pageType"] == 4:
      # Add video block
      page_video = page["video"]
      payload = {"operationName":"CreateVideo","variables":{"videoTitle":page_video["videoTitle"],"published":False,"saved":False,"videoUrl":page_video["videoUrl"],"videoType":page_video["videoType"],"thumbUrl":page_video["thumbUrl"]},"query":"mutation CreateVideo($videoTitle: String!, $published: Boolean, $saved: Boolean, $thumbUrl: String, $videoUrl: String, $videoType: VideoType!) {\n  createVideo(videoTitle: $videoTitle, published: $published, saved: $saved, thumbUrl: $thumbUrl, videoUrl: $videoUrl, videoType: $videoType) {\n    video {\n      id\n      videoTitle\n      videoUrl\n      thumbUrl\n      status\n      __typename\n    }\n    uploadLink\n    __typename\n  }\n}\n"}
      result = requests.post(INT_BASE_URL+"/graphql?op=CreateVideo", json=payload, headers=destination_auth_headers)
      video_result = result.json()
      if video_result.get("errors"):
        print(payload)
        print(video_result)
        raise ValueError(video_result["errors"][0]["message"])
      else:
        video_id = video_result["data"]["createVideo"]["video"]["id"]

        # Add page
        payload = {"operationName":"AddDocumentPage","variables":{"documentId":destination_template_id,"pageType":page_type,"pageNum":int(page["pageNum"]),"videoId":video_id},"query":"mutation AddDocumentPage($documentId: String!, $pageNum: Int!, $videoId: String, $blockId: String, $pageType: DocumentPageType, $defaultEditorRows: [CreateRowInput!], $fontSettings: InputFontSettings) {\n  addDocumentPage(documentPageInput: {documentId: $documentId, pageNum: $pageNum, videoId: $videoId, blockId: $blockId, pageType: $pageType, defaultEditorRows: $defaultEditorRows, fontSettings: $fontSettings}) {\n    id\n    thumbUrl\n    pageType\n    pageNum\n    video {\n      id\n      videoUrl\n      thumbUrl\n      __typename\n    }\n    contentLinkBlock {\n      buttonText\n      description\n      id\n      thumbUrl\n      thumbExtension\n      title\n      url\n      __typename\n    }\n    editorBlock {\n      ...editorBlockFragment\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment editorBlockFragment on EditorBlock {\n  id\n  entityId\n  userId\n  contentId\n  pdfPageCount\n  content {\n    ...editorBlockContentFragment\n    __typename\n  }\n  __typename\n}\n\nfragment editorBlockContentFragment on EditorContent {\n  id\n  displayName\n  fontSettings {\n    defaultFont {\n      ...fontFragment\n      __typename\n    }\n    headingsFont {\n      ...fontFragment\n      __typename\n    }\n    __typename\n  }\n  sections {\n    id\n    displayName\n    rows {\n      id\n      type\n      columnCount\n      cells {\n        id\n        width\n        type\n        nodes {\n          id\n          type\n          content\n          srcUrl\n          imageId\n          linkUrl\n          fields {\n            id\n            type\n            category\n            mergeKey\n            recipientId\n            userId\n            customName\n            value\n            inputSettings {\n              type\n              required\n              label\n              richTextLabel\n              helpText\n              connectedId\n              width\n              minValue\n              maxValue\n              placeholder\n              options\n              __typename\n            }\n            __typename\n          }\n          inputFieldSets {\n            ids\n            __typename\n          }\n          caption\n          heading\n          imageSize\n          imageAlign\n          align\n          url\n          videoId\n          imageTransforms {\n            translateX\n            translateY\n            scale\n            __typename\n          }\n          dividerType\n          dividerPadding\n          tableStyle {\n            borderColor\n            headerRowBackgroundColor\n            alternateRowColor\n            __typename\n          }\n          name\n          preCalculated\n          locked\n          pricingTableSections {\n            id\n            displayName\n            columns {\n              id\n              name\n              displayName\n              width\n              enabled\n              __typename\n            }\n            rows {\n              id\n              values {\n                columnId\n                value\n                __typename\n              }\n              discountFlatFee\n              taxFlatFee\n              optionalProductFieldId\n              variableQuantityFieldId\n              __typename\n            }\n            sectionSummary {\n              discount {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              tax {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              price {\n                value\n                enabled\n                flatFee\n                displayName\n                __typename\n              }\n              __typename\n            }\n            __typename\n          }\n          pricingTableSummaryValues {\n            displayName\n            discount {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            tax {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            price {\n              value\n              enabled\n              flatFee\n              displayName\n              __typename\n            }\n            __typename\n          }\n          currencySettings {\n            formatOptions {\n              currency\n              currencyDisplay\n              __typename\n            }\n            locale\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment fontFragment on UpsertFont {\n  fontFamily\n  displayName\n  sources {\n    fontStyle\n    fontWeight\n    unicodeRange\n    format\n    src\n    __typename\n  }\n  __typename\n}\n"}
        add_document_page(payload, destination_auth_headers)

# Get or create destination template id
def get_destination_template(source_template_name, destination):
  # Get destination templates
  payload = {"operationName":"LoadResources","variables":{"directory":"/","resourceType":"Template","searchTerms":[],"limit":100,"offset":0,"sortBy":"UpdatedAt","sortOrder":"DESC"},"query":"query LoadResources($directory: String, $resourceType: ResourceType, $searchTerms: [String!]!, $limit: Int!, $offset: Int!, $sortBy: ResourceSortAttributes!, $sortOrder: String!) {\n  resources(directory: $directory, resourceType: $resourceType, searchTerms: $searchTerms, limit: $limit, offset: $offset, sortBy: $sortBy, sortOrder: $sortOrder) {\n    resources {\n      ...resourceListFragment\n      statistics {\n        ...resourceListStatisticsFragment\n        __typename\n      }\n      __typename\n    }\n    totalCount\n    __typename\n  }\n}\n\nfragment resourceListFragment on Resource {\n  id\n  contentId\n  directory\n  resourceType\n  title\n  updatedAt\n  __typename\n}\n\nfragment resourceListStatisticsFragment on ResourceStatistics {\n  usedCount\n  __typename\n}\n"}
  result = requests.post(INT_BASE_URL+"/graphql?op=LoadResources", json=payload, headers=destination["header"])
  template_result = result.json()
  if template_result.get("errors"):
    print(payload)
    print(template_result)
  else:
    destination_template_id = None
    for template in template_result["data"]["resources"]["resources"]:
      if template["title"] == source_template_name:
        destination_template_id = template["contentId"]
        break

    if destination_template_id:
      return destination_template_id
    else:
      destination_template_id = create_destination_template(destination, source_template_name)
      return destination_template_id

def create_destination_template(destination, source_template_name):
      # Create new template with same name
      payload = {"operationName":"CreateDocument","variables":{"type":"template"},"query":"mutation CreateDocument($type: DocumentType!) {\n      createDocument(type: $type) {\n        id\n      }\n    }\n"}
      result = requests.post(INT_BASE_URL+"/graphql?op=CreateDocument", json=payload, headers=destination["header"])
      template_result = result.json()
      if template_result.get("errors"):
        print(payload)
        print(template_result)
        raise ValueError(template_result["errors"][0]["message"])
      else:
        template_id = template_result["data"]["createDocument"]["id"]
        payload = {"operationName":"UpdateDocument","variables":{"settings":{},"patch":{"name":source_template_name},"documentId":template_id},"query":"mutation UpdateDocument($documentId: String!, $settings: UpdateDocumentSettings = {transferMode: transfer}, $patch: UpdateDocumentPatch!) {\n  updateDocumentV2(documentId: $documentId, settings: $settings, patch: $patch)\n}\n"}
        result = requests.post(INT_BASE_URL+"/graphql?op=UpdateDocument", json=payload, headers=destination["header"])
        update_result = result.json()
        if update_result.get("errors"):
          print(payload)
          print(update_result)
          raise ValueError(update_result["errors"][0]["message"])
        else:
          payload = {"operationName":"CreateResource","variables":{"payload":{"title":source_template_name,"resourceType":"Template","resourceStatus":"Draft","contentId":template_id,"directory":"/"}},"query":"mutation CreateResource($payload: CreateResourcePayload!) {\n  createResource(payload: $payload) {\n    ...resourceDetailsFragment\n    __typename\n  }\n}\n\nfragment resourceDetailsFragment on Resource {\n  id\n  contentId\n  directory\n  resourceType\n  resourceStatus\n  title\n  user {\n    id\n    __typename\n  }\n  createdAt\n  updatedAt\n  deletedAt\n  __typename\n}\n"}
          result = requests.post(INT_BASE_URL+"/graphql?op=CreateResource", json=payload, headers=destination["header"])
          resource_result = result.json()
          if resource_result.get("errors"):
            print(payload)
            print(resource_result)
            raise ValueError(resource_result["errors"][0]["message"])
          else:
            resource_id = resource_result["data"]["createResource"]["id"]
            print("- Created new document \"" + template_id + "\" with resource id \"" + resource_id + "\"")
            return template_id
print("Ready")

In [ ]:
#@markdown Running copy-functions for each destination entity:
for i,template_id in enumerate(source_template_picker.value):
  template_name = source_template_picker.label[i]
  print("Copy template: \"" + template_name + "\"")
  source_pages, source_recipients, source_attachments, source_template_name = get_source_template(template_id)
  for dest in destination_entities:
    print("Destination entity: \"" + dest["name"] + "\":")
    # Get or create destination template
    destination_template_id = get_destination_template(source_template_name, dest)
    # Get destination template pages
    destination_pages, destination_recipients, destination_attachments = get_destination_template_data(destination_template_id, dest["header"])
    # Add template roles
    destination_recipients = sync_recipient_roles(source_recipients, destination_template_id, destination_recipients, dest["header"])
    # Add template attachments
    sync_attachments(source_attachments, destination_template_id, destination_attachments, dest["header"])
    # Delete previous pages
    delete_destination_pages(destination_template_id, destination_pages, dest["header"])
    # Add new pages from source
    copy_pages(source_pages, destination_template_id, destination_recipients, dest["header"])

print("All templates has been copied")